In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import lightgbm as lgb
from sklearn import preprocessing, pipeline, compose, model_selection, metrics
from typing import Callable
from scipy.sparse import hstack

from sales_forecasting.utils import timeseries_split, kfold_timeseries_split

In [4]:
df_train = pd.read_parquet(".data/df_train.parquet")
df_test = pd.read_parquet(".data/df_test.parquet")
df_full = pd.read_parquet(".data/df_full.parquet")
df_items = pd.read_csv(".data/items.csv")

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   shop_id             int64  
 1   item_id             int64  
 2   date_block_num      int64  
 3   date_month          int64  
 4   shop_name           object 
 5   item_name           object 
 6   item_category_id    int64  
 7   item_category_name  object 
 8   item_price          float64
 9   item_cnt_month      float64
dtypes: float64(2), int64(5), object(3)
memory usage: 224.0+ MB


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   shop_id             214200 non-null  int64  
 1   item_id             214200 non-null  int64  
 2   date_block_num      214200 non-null  int64  
 3   date_month          214200 non-null  int64  
 4   shop_name           214200 non-null  object 
 5   item_name           214200 non-null  object 
 6   item_category_id    214200 non-null  int64  
 7   item_category_name  214200 non-null  object 
 8   item_price          0 non-null       float64
 9   item_cnt_month      0 non-null       float64
dtypes: float64(2), int64(5), object(3)
memory usage: 16.3+ MB


In [ ]:
def reduce_mem_usage(df, silent=True, allow_categorical=True, float_dtype="float32"):
    """ 
    Iterates through all the columns of a dataframe and downcasts the data type
     to reduce memory usage. Can also factorize categorical columns to integer dtype.
    """
    def _downcast_numeric(series, allow_categorical=allow_categorical):
        """
        Downcast a numeric series into either the smallest possible int dtype or a specified float dtype.
        """
        if pd.api.types.is_sparse(series.dtype):
            return series
        elif not pd.api.types.is_numeric_dtype(series.dtype):
            if pd.api.types.is_datetime64_any_dtype(series.dtype):
                return series
            else:
                if allow_categorical:
                    return series
                else:
                    codes, uniques = series.factorize()
                    series = pd.Series(data=codes, index=series.index)
                    series = _downcast_numeric(series)
                    return series
        else:
            series = pd.to_numeric(series, downcast="integer")
            
        if pd.api.types.is_float_dtype(series.dtype):
            series = series.astype(float_dtype)
        return series

    if silent is False:
        start_mem = np.sum(df.memory_usage()) / 1024 ** 2
        print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    if df.ndim == 1:
        df = _downcast_numeric(df)
    else:
        for col in df.columns:
            df.loc[:, col] = _downcast_numeric(df.loc[:,col])
    if silent is False:
        end_mem = np.sum(df.memory_usage()) / 1024 ** 2
        print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
        print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df


def shrink_mem_new_cols(matrix, oldcols=None, allow_categorical=False):
    # Calls reduce_mem_usage on columns which have not yet been optimized
    if oldcols is not None:
        newcols = matrix.columns.difference(oldcols)
    else:
        newcols = matrix.columns
    matrix.loc[:,newcols] = reduce_mem_usage(matrix.loc[:,newcols], allow_categorical=allow_categorical)
    oldcols = matrix.columns  # This is used to track which columns have already been downcast
    return matrix, oldcols


def list_if_not(s, dtype=str):
    # Puts a variable in a list if it is not already a list
    if type(s) not in (dtype, list):
        raise TypeError
    if (s != "") & (type(s) is not list):
        s = [s]
    return s